In [ ]:
Show DA at WFJ compared to sim driven by station data.
/home/joel/mnt/myserver/nas/sim/gcos_era5_big_ensemble


In [1]:
# dependencies
source("/home/joel/src/topoMAPP/rsrc/PBS.R")
require(raster)

Loading required package: raster
Loading required package: sp


In [2]:
# define rmse
rmse = function(sim, obs){sqrt( mean( (sim - obs)^2, na.rm = TRUE) )  }

In [5]:
# define variable
lwd=3
priorwd="/home/joel/mnt/myserver/nas/sim/gcos_era5_big"
wd = "/home/joel/mnt/myserver/nas/sim/gcos_era5_big_ensemble"
grid= 56
year = 0
startda = "2015-09-01"#(yyy-mm-dd)
endda = "2016-09-01"
startSim = "2015-09-01"#(yyy-mm-dd)
endSim = "2016-09-01"
sdThresh=13
createPixTS="TRUE"
start=300#dohy for obs in da 
end=340 #dohy for obs in da 
nens=100
R=0.016

In [9]:
# read in stuff
landform = raster(paste0(priorwd, "/grid", grid, "/landform.tif"))
rstack = brick(paste0(priorwd, "/fsca_stack.tif"))
obsTS = read.csv(paste0(priorwd, "/fsca_dates.csv"))
load(paste0(wd, "/ensembRes_", grid, ".rd"))
gcosshp = shapefile('/home/joel/mnt/myserver/nas/data/GCOS/metadata_easy.shp')

In [10]:

dshp = crop(gcosshp,landform)
print(c("stations found = ", dshp$STAT_AB))
statn = length(dshp$STAT_AB)

[1] "stations found = " "5DF"               "5KK"              
[4] "5WJ"              


In [12]:
# read in val data
valshp = dshp[3,] # options 1=5DF, 2=5KK, 3=5WJ
gcosName=valshp$STAT_AB
valdat = read.csv(paste0("/home/joel/mnt/myserver/nas/data/GCOS/sp_",gcosName,".txt"))

In [13]:
# readin or crop MODIS fsca domain to grid
# ensure it is gfap filled temporally

fscacrop = paste0(wd, "/fsca_crop",grid,year,".tif")
if (!file.exists(fscacrop)) {

    # cut temporal length of dates vector to startda/endda
    startda.index <- which(obsTS$x == startda)
    endda.index <- which(obsTS$x == endda)

    # subset rstack temporally
    print(paste0("subset rstack temporally:", startda," to ",endda))
    rstack = rstack[[startda.index:endda.index]]

    # subset dates vector to current year
    obsTScut <- obsTS$x[startda.index:endda.index]
    write.csv(obsTScut, paste0(wd, "/fsca_dates_",year,".csv"), row.names = FALSE)

    # analyse missing days
    actualDays <- seq(as.Date(startda), as.Date(endda), 1)
    NactualDays <- length(actualDays)

    # check for missing dates in fsca timeseries (not NA but actual missing
    # timestamps) THIS was implement in extractSCATimeseries.R but not in _GRID.R
    # version - fill with dummy NA layers
    if (nlayers(rstack) != NactualDays) {
        
        missDates.index <- which(!(as.character(actualDays) %in% as.character(obsTScut)))
        missDates <- actualDays[which(!(as.character(actualDays) %in% as.character(obsTScut)))]

        print(paste0("missing dates found:", missDates, " filling missing datestamps now"))

        # create NA layer for insertion
        insertLyr = rstack[[1]]
        insertLyr[insertLyr < 1000] <- NA

        # name layers by date
        names(rstack) <- obsTScut

        # make stack of dummy layes and name
        insertStack <- stack(replicate(length(missDates), insertLyr))
        names(insertStack) <- missDates
        mergeStack = stack(rstack, insertStack)
        rstack = mergeStack[[order(names(mergeStack))]]
        print("rstack missing datestamps filled")

    }
    } else {
    print(paste0(fscacrop, " already exists."))
    rstack <- stack(fscacrop)
}


[1] "/home/joel/mnt/myserver/nas/sim/gcos_era5_big_ensemble/fsca_crop560.tif already exists."


In [14]:
# GET FSCA TIMESERIES AT VALPOINT LOCATION

npix = cellsFromExtent(rstack[[1]], valshp)
print(paste0("Validation data exists at pixels: ", npix))

# OPTION1 - use this to create file
if (createPixTS==TRUE){
pixTS = as.numeric(extract(rstack, valshp))
}


# OPTION2 - use this to read file already created, WARNING! make sure it isnt fro another sim.

if (createPixTS==FALSE){
pixTS = paste0(wd, "/pixTS_", grid)
if (!file.exists(pixTS)) {

   pixTS = as.numeric(extract(rstack, valshp))
    save(pixTS, file = paste0(wd, "/pixTS_", grid))

} else {
    print(paste0(pixTS, " already exists."))
    load(paste0(wd, "/pixTS_", grid))
}
}

[1] "Validation data exists at pixels: 1142"


In [15]:
# GET ENSEMBLE SUBSET AND CONVERT TO SCA

# subset temporally
totalTS <- seq(as.Date(startSim), as.Date(endSim), 1)
start.index <- which(totalTS == startda)
end.index <- which(totalTS == endda)
ensembRes <- ensembRes[start.index:end.index, , ]
print(paste0("ensembRes cut to: ", startda, " to ", endda))

# convert swe to sca
ensembRes[ensembRes <= sdThresh] <- 0
ensembRes[ensembRes > sdThresh] <- 1

[1] "ensembRes cut to: 2015-09-01 to 2016-09-01"


In [16]:
# COMPUTE meltperiod

vec <- pixTS
   vec[1:150] <-NA
        lastdata = which(vec == 0)[1]  # last non-zero value
    
       
        #lastdateover95 = length(vec) - which(rvec > (max(rvec, na.rm = TRUE) * 0.95))[1]  # last date over 95% of max value accounts for max below 100%
        start = lastdata - 30
        end = lastdata +5
        
print(paste0("meltperiod start: ", start))
print(paste0("meltperiod end: ", end))

[1] "meltperiod start: 281"
[1] "meltperiod end: 316"
